# Fine-tuning Question Answering Model

This exam will guide you through loading, preprocessing, and fine-tuning a pre-trained model for a question-answering task using a dataset. Follow the steps carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `distilbert-base-cased` for both the model and tokenizer.
- **Dataset**: You will be using the `christti/squad-augmented-v2` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
! pip install datasets
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [36]:
from datasets import load_dataset


dataset = load_dataset("christti/squad-augmented-v2", split='train[:1000]')

dataset_split = dataset.train_test_split(test_size=0.2)

train_data = dataset_split['train']
test_data = dataset_split['test']


print("Training data size:", len(train_data))
print("Test data size:", len(test_data))


Training examples: 800, Testing examples: 200


## Step 2: Load the Pretrained Tokenizer and Model

In [37]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Use the model and tokenizer for the question-answering task.

## Step 3: Preprocess the Dataset

Define a function to preprocess the dataset by tokenizing both the context and the question. The function will also calculate the start and end positions of the answers. In the tokenizer you might face a problem if you use `truncation=True` so consider using `truncation='only_first'` if needed.

In [51]:
def preprocess_data(examples):
    inputs = tokenizer(
        examples['context'],
        examples['question'],
        padding='max_length',
        max_length=384,
        truncation='only_first'
    )

    answers = examples['answers']
    start_positions = []
    end_positions = []

    for i, answer in enumerate(answers):
        answer_text = answer['text'][0]
        start_char = answer['answer_start'][0]

        context = examples['context'][i]
        tokenized_context = tokenizer(context, truncation='only_first', padding='max_length', max_length=384)


        tokenized_answer = tokenizer(answer_text, truncation='only_first', padding='max_length', max_length=384)


        start_pos = None
        end_pos = None


        for idx in range(len(tokenized_context['input_ids']) - len(tokenized_answer['input_ids']) + 1):
            if tokenized_context['input_ids'][idx:idx + len(tokenized_answer['input_ids'])] == tokenized_answer['input_ids']:
                start_pos = idx
                end_pos = idx + len(tokenized_answer['input_ids']) - 1
                break

        if start_pos is None or end_pos is None:
            start_pos = 0
            end_pos = 0

        start_positions.append(start_pos)
        end_positions.append(end_pos)

    inputs.update({
        'start_positions': start_positions,
        'end_positions': end_positions
    })

    return inputs
train_data = dataset.map(preprocess_data, batched=True, remove_columns=['id', 'title', 'context', 'question', 'answers'])


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Step 4: Define Training Arguments and Initialize the Trainer

In [46]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
evaluation_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=9,
    num_train_epochs=5
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
trainer.train()

Step,Training Loss
500,0.000000


TrainOutput(global_step=560, training_loss=2.689356603825997e-05, metrics={'train_runtime': 205.6714, 'train_samples_per_second': 24.311, 'train_steps_per_second': 2.723, 'total_flos': 489949125120000.0, 'train_loss': 2.689356603825997e-05, 'epoch': 5.0})

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

## Step 5: Fine-tune the Model

Run the training process using the initialized trainer.

## Step 6: Inference

Once the model is trained, perform inference by answering a question based on a context. Use the tokenizer to process the input, and then feed it into the model to get the predicted answer.

In [50]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="/content/results/checkpoint-125",
    tokenizer="distilbert-base-cased",
)

context = """
Elephants are the largest land animals, known for their trunks and tusks. They are native to Africa and Asia
,the zarafa are tall.
"""

question =" What is the largest land animal?"
question2 ="Where are elephants originally from?"

result = qa_pipeline({
    'context': context,
    'question': question

})
result2 = qa_pipeline({
    'context': context,
    'question': question2

})



print("Answer:", result['answer'])
print('---------------------------------------')

print("Answer:", result2['answer'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Answer: Elephants
---------------------------------------
Answer: Africa and Asia
